In [1]:
import numpy as np
import gurobipy as gbp
import time
np.random.seed(352)


In [20]:
n_com=1
n_sup=2
n_dem=3

In [8]:
Supply = np.random.randint(50,200,(n_sup,n_com))   # Vector of Units Supplied
Demand = np.random.randint(25,30,(n_dem,n_com))    # Vector of Units Demanded
Supply,Demand

(array([[ 59, 130, 190, 120],
        [156, 131, 177,  65]]), array([[27, 28, 25, 29],
        [28, 29, 26, 25],
        [26, 28, 27, 25]]))

In [23]:
Supply=np.array([[100],[200]])
Demand=np.array([[75],[150],[200]])
Supply,Demand

(array([[100],
        [200]]), array([[ 75],
        [150],
        [200]]))

In [ ]:
# matrix_rows = len(Supply)                # Supply (i) x Demand (j)
# matrix_cols = len(Demand)

# Cost_Matrix = np.random.randint(3, 50, 
#                                 matrix_rows*matrix_cols)
# Call Function   


In [24]:
Si=Supply
Di=Demand

In [25]:
# Cij = Cij.reshape(matrix_rows,matrix_cols)
# Si = Si.reshape(matrix_rows,1)
# Dj = Dj.reshape(matrix_cols,1)

# Indices & Variable Names
supply_nodes = n_sup
demand_nodes = n_dem
supply_nodes_range = range(n_sup)
demand_nodes_range = range(n_dem)
all_nodes_len = n_sup*n_dem
ALL_nodes_range = range(all_nodes_len)

print (supply_nodes_range, demand_nodes_range, all_nodes_len)

range(0, 2) range(0, 3) 6


In [ ]:
gbp.ParamClass.

In [184]:
# Create Model, Set MIP Focus, Add Variables, & Update Model
m = gbp.Model(' -- The Transportation Problem -- ')
# Set MIP Focus to 2 for optimality
m.setParam('MIPFocus', 1)
m.setParam(gbp.GRB.Param.PoolSearchMode, 1)
m.setParam(gbp.GRB.Param.PoolGap, 0.10)
decision_var = []
for orig in supply_nodes_range:
    decision_var.append([])
    for dest in demand_nodes_range:
        decision_var[orig].append(m.addVar(vtype=gbp.GRB.INTEGER, 
#                                         obj=Cij[orig][dest],
#                                            obj=1,
                                        name='S'+str(orig+1)+'_D'+str(dest+1)))
# Update Model Variables
m.update()       


Parameter MIPFocus unchanged
   Value: 1  Min: 0  Max: 3  Default: 0
Parameter PoolSearchMode unchanged
   Value: 1  Min: 0  Max: 2  Default: 0
Changed value of parameter PoolGap to 0.1
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [185]:
m.setObjective(gbp.quicksum(int(Di[dest])-gbp.quicksum(decision_var[orig][dest] for orig in supply_nodes_range)
                            for dest in demand_nodes_range), 
                        gbp.GRB.MINIMIZE)

In [186]:
m.update()

In [187]:
m.display()

Minimize
   <gurobi.LinExpr: 425.0 + -1.0 S1_D1 + -1.0 S1_D2 + -1.0 S1_D3 + -1.0 S2_D1 + -1.0 S2_D2 + -1.0 S2_D3>
Subject To
General Integers
   ['S1_D1', 'S1_D2', 'S1_D3', 'S2_D1', 'S2_D2', 'S2_D3']


In [188]:
# Add Supply Constraints
for orig in supply_nodes_range:
    m.addConstr(gbp.quicksum(decision_var[orig][dest] 
                        for dest in demand_nodes_range) - Si[orig] <= 0)
# Add Demand Constraints
for dest in demand_nodes_range:  
    m.addConstr(gbp.quicksum(decision_var[orig][dest] 
                        for orig in supply_nodes_range) - Di[dest] <= 0)

#  Optimize and Print( Results)
m.optimize()
m.write('path.lp')

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 5 rows, 6 columns and 12 nonzeros
Model fingerprint: 0xb4f257f5
Variable types: 0 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+01, 2e+02]
Found heuristic solution: objective 125.0000000
Presolve time: 0.00s
Presolved: 5 rows, 6 columns, 12 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)

Root relaxation: objective 1.250000e+02, 5 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0          -    0       125.00000  125.00000  0.00%     -    0s

Optimal solution found at node 0 - now completing solution pool...

    Nodes    |    Current Node    |      Pool Obj. Bounds     |     Work
             |                    |   Worst     

In [189]:
nSolutions = m.SolCount
print (nSolutions)
for e in range(nSolutions):
        m.setParam(gbp.GRB.Param.SolutionNumber, e)
        print('%g ' % m.PoolObjVal, end='')
        print ([(v.Varname,v.x) for v in m.getVars()])

200
125 [('S1_D1', 75.0), ('S1_D2', 25.0), ('S1_D3', -0.0), ('S2_D1', -0.0), ('S2_D2', 125.0), ('S2_D3', 75.0)]
125 [('S1_D1', 75.0), ('S1_D2', 25.0), ('S1_D3', -0.0), ('S2_D1', -0.0), ('S2_D2', 125.0), ('S2_D3', 75.0)]
125 [('S1_D1', 75.0), ('S1_D2', 25.0), ('S1_D3', -0.0), ('S2_D1', -0.0), ('S2_D2', 125.0), ('S2_D3', 75.0)]
125 [('S1_D1', 75.0), ('S1_D2', 25.0), ('S1_D3', -0.0), ('S2_D1', -0.0), ('S2_D2', 125.0), ('S2_D3', 75.0)]
125 [('S1_D1', 75.0), ('S1_D2', 25.0), ('S1_D3', -0.0), ('S2_D1', -0.0), ('S2_D2', 125.0), ('S2_D3', 75.0)]
125 [('S1_D1', 75.0), ('S1_D2', 25.0), ('S1_D3', -0.0), ('S2_D1', -0.0), ('S2_D2', 125.0), ('S2_D3', 75.0)]
125 [('S1_D1', 75.0), ('S1_D2', 25.0), ('S1_D3', -0.0), ('S2_D1', -0.0), ('S2_D2', 125.0), ('S2_D3', 75.0)]
125 [('S1_D1', 75.0), ('S1_D2', 25.0), ('S1_D3', -0.0), ('S2_D1', -0.0), ('S2_D2', 125.0), ('S2_D3', 75.0)]
125 [('S1_D1', 75.0), ('S1_D2', 25.0), ('S1_D3', -0.0), ('S2_D1', -0.0), ('S2_D2', 125.0), ('S2_D3', 75.0)]
125 [('S1_D1', 75.0), ('

In [ ]:
if (nSolutions >= 4):
    model.setParam(GRB.Param.SolutionNumber, 3)

    print('Selected elements in fourth best solution:')
    print('\t', end='')
    for e in range(2):
        if Elem[e].Xn > .9:
            print(' El%d' % e, end='')
    print('')

In [159]:
selected = {}
Closed = []
for v in m.getVars():
    var = '%s' % v.VarName
    print (v)
#     print ('|  Supply Facility #', var[1:6], 'is shipping', units, \
#                                             'units to Demand Facility #', var[-5:])

<gurobi.Var S1_D1 (value 75.0)>
<gurobi.Var S1_D2 (value 25.0)>
<gurobi.Var S1_D3 (value -0.0)>
<gurobi.Var S2_D1 (value -0.0)>
<gurobi.Var S2_D2 (value 125.0)>
<gurobi.Var S2_D3 (value 75.0)>


In [ ]:
selected = {}
Closed = []
for v in m.getVars():
    var = '%s' % v.VarName
    if v.x > 0:
        units = '%i' % v.x
        selected[var] = units
        print( '|  Supply Facility #', var[1:6], 'is shipping', units, \
                                            'units to Demand Facility #', var[-5:])
    else:
        Closed.append([var[1:6], var[-5:]])
print( '******************************************************************************')
print( '    | Objective Value --------------------- ', int(m.objVal))
print( '    | Supply Facilities ------------------- ', len(Si))
print( '    | Total Supply Units ------------------ ', Si.sum())
print( '    | Demand Facilites -------------------- ', len(Dj))
print( '    | Total Demand Units ------------------ ', Dj.sum())
print( '    | Total Potential Combinations -------- ', len(Si)*len(Dj))
print( '    | Actual Combinations  ---------------- ', len(selected))
# print( '    | Real Time to Optimize (sec.) -------- ', t2)
print( '******************************************************************************')
print( '  --  The Transportation Simplex with Gurobi --')
